### PLEASE comment/document what each function is doing in your own words to demonstrate your understanding. Of course, push it up to your GitHub on completion.

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_hastie_10_2
import matplotlib.pyplot as plt


In [4]:
""" HELPER FUNCTION: GET ERROR RATE ========================================="""
# we define new funcions, with arguments pred (prediction) and Y
# sums all predictions which 
#are not equal to actual Y values, and divide it by the length of Y)
def get_error_rate(pred, Y):
    return sum(pred != Y) / float(len(Y))

""" HELPER FUNCTION: PRINT ERROR RATE ======================================="""
# print the error generated in above function with 4 figures after dot
def print_error_rate(err):
    print ('Error rate: Training: %.4f - Test: %.4f' % err)

""" HELPER FUNCTION: GENERIC CLASSIFIER ====================================="""
# we define a function to take an arbitrary classifier
# fit it to the training data
# predixt classification for train and test
# and return the error rates (train and test) by implementing the above functions
def generic_clf(Y_train, X_train, Y_test, X_test, clf):
    clf.fit(X_train,Y_train)
    pred_train = clf.predict(X_train)
    pred_test = clf.predict(X_test)
    return get_error_rate(pred_train, Y_train), \
           get_error_rate(pred_test, Y_test)

In [3]:
""" ADABOOST IMPLEMENTATION ================================================="""
def adaboost_clf(Y_train, X_train, Y_test, X_test, M, clf):
    # defines the lenght of data sets
    n_train, n_test = len(X_train), len(X_test)
    # Initialize weights, creates an array of one, divided by the lenght of n_train
    w = np.ones(n_train) / n_train
    # bereitet die arrays, and fills with zeros
    pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]
    
    for i in range(M):
        # Fit a classifier with the specific weights
        # and predict for train and test
        clf.fit(X_train, Y_train, sample_weight = w)
        pred_train_i = clf.predict(X_train)
        pred_test_i = clf.predict(X_test)
        # Indicator function
        # creates a list with all items which dont correspond to 
        # Y_train, and uses int to covert True/False into 1/0
        # values are 1 if it does not correspond and 0 if it does
        miss = [int(x) for x in (pred_train_i != Y_train)]
        # Equivalent with 1/-1 to update weights
        # creates a new list miss2 for all items in miss
        # with values 1 for 1 and -1 for zero
        # 1 is false predicted and -1 for correct predicted
        miss2 = [x if x==1 else -1 for x in miss]
        # multiplicates each value of w (e,g. 1/8 for n_train_len = 8) 
        # with miss and divides by sum of weights
        # np.dot is "scalierprodukt" which multiplies the values of two vectors
        # addiert weight wenns falsch vorhergesagt wurde
        # sum(w)==0 in erstem Loop
        err_m = np.dot(w,miss) / sum(w)
        # alpha_m provides "amount of say" (wight) value for the classifier
        # to be used at the end of the ensemble prediction
        # np.log function returns large positive nubmer if err_m is small (i.e. err_m < 0.5)
        # np.log function returns large negative nubmer if err_m is large (i.e. err_m > 0.5)
        # np.log function returns zeor nubmer if err_m is 0.5
        alpha_m = 0.5 * np.log( (1 - err_m) / float(err_m))
        # New weights
        # if x == 1.0 (false predicted), and 
        # alpha_m == e.g.5 (good predictor = high "amount of say")
        # then we take an e^(1*5) (== which is a high number)
        # and multipy it with old weight (e.g. 1/8)
        # which is now our new weight for the instance
        # wheight will be larger for false prediction
        # 
        # and other wise if x == -1.0 (correct prediction)
        # alpha_m == e.g. 5, then take an e^(-1*5) (== which is a low number)
        # and multipy it with old weight (e.g. 1/8)
        # which is now our new weight for the instance
        # wheight will be smaller for correct prediction
        w = np.multiply(w, np.exp([float(x) * alpha_m for x in miss2]))
        # Add to prediction
        # updates old prediction with product of new prediction and alpha_m
        # double list comprehension
        pred_train = [sum(x) for x in zip(pred_train, 
                                          [x * alpha_m for x in pred_train_i])]
        pred_test = [sum(x) for x in zip(pred_test, 
                                         [x * alpha_m for x in pred_test_i])]
    
    # the values in pred_train and pred_test are transfomred in 1, if x >0
    # and in -1, if x<0
    pred_train, pred_test = np.sign(pred_train), np.sign(pred_test)
    # Return error rate in train and test set
    return get_error_rate(pred_train, Y_train), \
           get_error_rate(pred_test, Y_test)

In [4]:

""" PLOT FUNCTION ==========================================================="""
def plot_error_rate(er_train, er_test):
    df_error = pd.DataFrame([er_train, er_test]).T
    df_error.columns = ['Training', 'Test']
    plot1 = df_error.plot(linewidth = 3, figsize = (8,6),
            color = ['lightblue', 'darkblue'], grid = True)
    plot1.set_xlabel('Number of iterations', fontsize = 12)
    plot1.set_xticklabels(range(0,450,50))
    plot1.set_ylabel('Error rate', fontsize = 12)
    plot1.set_title('Error rate vs number of iterations', fontsize = 16)
    plt.axhline(y=er_test[0], linewidth=1, color = 'red', ls = 'dashed')

